# Pertemuan 15

- Load Preprocessing & Feature Extraction Class as Module
- Mockup Application
- Layouting Preprocessing
- Layouting Feature Extraction

In [1]:
import os
import cv2
import json
import numpy as np
import PySimpleGUI as sg 
import matplotlib.pyplot as plt

## 1. Load Preprocessing & Feature Extraction Class as Module

- File `CoreService.py` menyimpan Class `Preprocessing()` dan `FeatureExtration()`
- Dua class tersebut dapat digunakan dengan cara import module (seperti import library)

In [2]:
from CoreService import Preprocessing
from CoreService import FeatureExtraction

- Create Object Preprocessing & Run Preprocessing Data

In [34]:
Prepro = Preprocessing(DATASET_FOLDER = "Dataset_Tomat/")

In [35]:
Prepro.ImageRead()
Prepro.RangeTresholding()
Prepro.EdgeDetection()
Prepro.FindContour()
Prepro.FilterContour()

In [36]:
Prepro.CropByContour()
Prepro.Resize()

In [37]:
Prepro.SaveAllImage(RESIZED_FOLDER = "resized_tomato/")

- Create Object Feature Extraction & Run Feature Extration Data

In [38]:
Feature = FeatureExtraction(PREPROCESSED_DATASET_FOLDER = "resized_tomato/")

In [39]:
Feature.ImageRead()
Feature.CalcStatisticalFeature()

In [40]:
Feature.CalcGLCMMatrix()
Feature.CalcGLCMTextureFeature()

- Exception / Error Handling

In [37]:
a = int(input("a:"))

try :
    b = 1/a
    print("1/%d=%.2f" % (a, b))
except :
    print("error operation!")


a: 0


error operation!


- System Error Message

In [39]:
a = int(input("a:"))

try :
    b = 1/a
    print("1/%d=%.2f" % (a, b))
except Exception as e:
    print("[ERROR] %s" % e)


a: 0


[ERROR] division by zero


- Problem Layout as varible in Jupyter Lab 

In [47]:
Layout = [[sg.Text("Hello World!")]]

In [48]:
window = sg.Window("My Application", layout=Layout, margins=(200, 100))
window.read()
window.close()

- Layout as Function

In [49]:
def Layout():
    return [[sg.Text("Hello World!")]]

In [51]:
window = sg.Window("My Application", layout=Layout(), margins=(200, 100))
window.read()
window.close()

## 2. Mockup Application

- **Screen Preprocessing** <br>
<img src="resource/Screen4.png"></img>

- **Screen Feature Extraction** <br>
<img src="resource/Screen3.png"></img>

- **Screen Training Model** <br>
<img src="resource/Screen2.png"></img>

- **Screen Predict Data** <br>
<img src="resource/Screen1.png"></img>

## 3. Layouting Preprocessing

In [52]:
# Layout Predict Data
def layout_PredictData():
    return [[sg.Text("Under Development")]]

In [53]:
# Layout Train Model
def layout_TrainingModel():
    return [[sg.Text("Under Development")]]

In [54]:
# Layout Feature Extraction
def layout_FeatureExtraction():
    return [[sg.Text("Under Development")]]

In [55]:
# Layout Preprocessing
def layout_Preprocessing_left():
    return [
                [
                    sg.Text("Dataset Folder", key="label_browse"),
                    sg.In(size=(25, 1), enable_events=True, key="Dataset_Folder"),
                    sg.FolderBrowse(),
                ],
                [
                    sg.Listbox(values=[], enable_events=True, size=(40, 20), key="File_List")
                ],
            ]

def layout_Preprocessing_right():
    return [
                [sg.Button("Run Preprocessing", key="preprocessing"), sg.Button("  Reset  ", key="reset", visible=False)],
                [sg.Image(filename='empty.png', key="image_data")]
            ]

def layout_Preprocessing(): 
    return [
                [
                    sg.Column(layout_Preprocessing_left()),
                    sg.VSeperator(),
                    sg.Column(layout_Preprocessing_right()),
                ]
            ]

In [56]:
# Main Layout

def Layout():
    return [[sg.TabGroup([
                    [sg.Tab('Predict Data', layout_PredictData()), 
                     sg.Tab('Training Model', layout_TrainingModel()),
                     sg.Tab('Feature Extraction', layout_FeatureExtraction()), 
                     sg.Tab('Preprocessing', layout_Preprocessing())]
                    ])
           ]]

In [57]:
# Funtion Convert Matrix Img to Byte
def ImgToByte(filename):
    img = cv2.imread(filename) 
    ret, frame_png = cv2.imencode('.png', img)
    img_byte = frame_png.tobytes()
    return img_byte

In [58]:
def GetFileList(root_folder):
    # baca seluruh file & folder untuk root folder yang di browse
    file_paths = []
    for folder in os.listdir(root_folder):
        for file in os.listdir(os.path.join(root_folder, folder)):
            path = os.path.join(folder, file)
            file_paths.append(path)
    return file_paths

In [59]:
# Main Program
PREPROCESSED_FOLDER = "resized_tomato/"

window = sg.Window("Tomato Grading Systems", layout=Layout(), margins=(10, 10), finalize=True)
window.set_min_size((800,500))

while True:
    event, values = window.read(timeout=25)
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
        
    if event == "Dataset_Folder":
        # baca root folder yang di browse
        root_folder = values["Dataset_Folder"]
        
        file_paths = GetFileList(root_folder)
        
        # update Listbox widget dengan list nama-nama file yang terbaca
        window["File_List"].update(file_paths)
        
    elif event == "File_List":  
        root_folder = values["Dataset_Folder"]
        path = values["File_List"][0]
        file_path = os.path.join(root_folder, path).replace("\\","/")
        img_byte = ImgToByte(filename = file_path)
     
        window['image_data'].update(data=img_byte, size=(500,400)) 
        
    elif event == "preprocessing" :
        root_folder = values["Dataset_Folder"]
        if root_folder == "":
            sg.popup("Please Select Dataset Folder First!") # show Popup if Dataset not selected
        else :
            Prepro = Preprocessing(DATASET_FOLDER = root_folder)
            Prepro.ImageRead()
            Prepro.RangeTresholding()
            Prepro.EdgeDetection()
            Prepro.FindContour()
            Prepro.FilterContour()
            Prepro.CropByContour()
            Prepro.Resize()
            PREPROCESSED_PATH = os.path.join(os.path.dirname(root_folder), PREPROCESSED_FOLDER).replace("\\","/")
            Prepro.SaveAllImage(RESIZED_FOLDER = PREPROCESSED_PATH) 
            sg.popup("Preprocessed data saved in folder %s" % PREPROCESSED_FOLDER) # show Popup to notify preprocessed data saved successfully!
            
            # Cleanup window
            window['image_data'].update(filename="empty.png")
            window['Dataset_Folder'].update(value=PREPROCESSED_PATH)
        
            file_paths = GetFileList(PREPROCESSED_PATH)
            window['File_List'].update(file_paths)
            window['label_browse'].update(value="Preprocessed Folder")
            window['preprocessing'].update(visible=False)
            window['reset'].update(visible=True)
        
window.close()

## 4. Layouting Feature Extraction

- Ubah function `layout_FeatureExtraction()` diatas menjadi seperti berikut,

In [3]:
# Layout Predict Data
def layout_PredictData():
    return [[sg.Text("Under Development")]]

In [4]:
# Layout Train Model
def layout_TrainingModel():
    return [[sg.Text("Under Development")]]

In [26]:
# Layout Feature Extraction
def layout_FeatureExtraction_left():
    return [
                [
                    sg.Text("Preprocessed Folder", key="preprocess_label"),
                    sg.In(size=(25, 1), enable_events=True, key="Preprocess_Folder"),
                    sg.FolderBrowse(),
                ],
                [
                    sg.Listbox(values=[], enable_events=True, size=(40, 20), key="Preprocess_File_List")
                ],
            ]

def layout_FeatureExtraction_right():
    return [
                [sg.Button("Run Feature Extraction", key="feature_extraction"), sg.Button("  Reset  ", key="feature_reset", visible=False)],
                [
                    sg.Column([[
                                sg.Text("<no feature to show>", 
                                        key="feature_output", 
                                        size=(50,100), 
                                        background_color="#f0f3f7", 
                                        text_color="#000000")
                            ]], scrollable=True, vertical_scroll_only=True, size=(400,350))
                ]
            ]

def layout_FeatureExtraction(): 
    return [
                [
                    sg.Column(layout_FeatureExtraction_left()),
                    sg.VSeperator(),
                    sg.Column(layout_FeatureExtraction_right()),
                ]
            ]

In [27]:
# Layout Preprocessing
def layout_Preprocessing_left():
    return [
                [
                    sg.Text("Dataset Folder", key="label_browse"),
                    sg.In(size=(25, 1), enable_events=True, key="Dataset_Folder"),
                    sg.FolderBrowse(),
                ],
                [
                    sg.Listbox(values=[], enable_events=True, size=(40, 20), key="File_List")
                ],
            ]

def layout_Preprocessing_right():
    return [
                [sg.Button("Run Preprocessing", key="preprocessing"), sg.Button("  Reset  ", key="reset", visible=False)],
                [sg.Image(filename='empty.png', key="image_data")]
            ]

def layout_Preprocessing(): 
    return [
                [
                    sg.Column(layout_Preprocessing_left()),
                    sg.VSeperator(),
                    sg.Column(layout_Preprocessing_right()),
                ]
            ]

In [28]:
# Main Layout

def Layout():
    return [[sg.TabGroup([
                    [sg.Tab('Predict Data', layout_PredictData()), 
                     sg.Tab('Training Model', layout_TrainingModel()),
                     sg.Tab('Feature Extraction', layout_FeatureExtraction()), 
                     sg.Tab('Preprocessing', layout_Preprocessing())]
                    ])
           ]]

In [29]:
# Funtion Convert Matrix Img to Byte
def ImgToByte(filename):
    img = cv2.imread(filename) 
    ret, frame_png = cv2.imencode('.png', img)
    img_byte = frame_png.tobytes()
    return img_byte

In [30]:
def GetFileList(root_folder):
    # baca seluruh file & folder untuk root folder yang di browse
    file_paths = []
    for folder in os.listdir(root_folder):
        for file in os.listdir(os.path.join(root_folder, folder)):
            path = os.path.join(folder, file)
            file_paths.append(path)
    return file_paths

In [33]:
# Main Program
global Feature
PREPROCESSED_FOLDER = "resized_tomato/"
window = sg.Window("Tomato Grading Systems", layout=Layout(), margins=(10, 10), finalize=True)
window.set_min_size((800,450))

while True:
    event, values = window.read(timeout=25)
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
        
    # Event Handler Preprocessing
    if event == "Dataset_Folder":
        # baca root folder yang di browse
        root_folder = values["Dataset_Folder"]
        
        file_paths = GetFileList(root_folder)
        
        # update Listbox widget dengan list nama-nama file yang terbaca
        window["File_List"].update(file_paths)
        
    elif event == "File_List":  
        root_folder = values["Dataset_Folder"]
        path = values["File_List"][0]
        file_path = os.path.join(root_folder, path).replace("\\","/")
        img_byte = ImgToByte(filename = file_path)
     
        window['image_data'].update(data=img_byte, size=(500,400)) 
        
    elif event == "preprocessing" :
        root_folder = values["Dataset_Folder"]
        if root_folder == "":
            sg.popup("Please Select Dataset Folder First!") # show Popup if Dataset not selected
        else :
            Prepro = Preprocessing(DATASET_FOLDER = root_folder)
            Prepro.ImageRead()
            Prepro.RangeTresholding()
            Prepro.EdgeDetection()
            Prepro.FindContour()
            Prepro.FilterContour()
            Prepro.CropByContour()
            Prepro.Resize()
            PREPROCESSED_PATH = os.path.join(os.path.dirname(root_folder), PREPROCESSED_FOLDER).replace("\\","/")
            Prepro.SaveAllImage(RESIZED_FOLDER = PREPROCESSED_PATH) 
            sg.popup("Preprocessed data saved in folder %s" % PREPROCESSED_FOLDER) # show Popup to notify preprocessed data saved successfully!
            
            # Cleanup window
            window['image_data'].update(filename="empty.png")
            window['Dataset_Folder'].update(value=PREPROCESSED_PATH)
        
            file_paths = GetFileList(PREPROCESSED_PATH)
            window['File_List'].update(file_paths)
            window['label_browse'].update(value="Preprocessed Folder")
            window['preprocessing'].update(visible=False)
            window['reset'].update(visible=True)
            
    # Event Handler Feature Extrcation
    elif event == "Preprocess_Folder":
        # baca root folder yang di browse
        root_folder = values["Preprocess_Folder"]
        
        file_paths = GetFileList(root_folder)
        
        # update Listbox widget dengan list nama-nama file yang terbaca
        window["Preprocess_File_List"].update(file_paths)
        
    elif event == "Preprocess_File_List":  
        root_folder = values["Preprocess_Folder"]
        path = values["Preprocess_File_List"][0]
        try :
            file_paths = GetFileList(root_folder)
            idx = file_paths.index(path)

            feature_stat = json.dumps(Feature.statistical_features[idx], indent=4, sort_keys=True)
            feature_glcm = json.dumps(Feature.glcm_feature_list[idx], indent=4, sort_keys=True)

            feature = "*"*30 + "\n" + feature_stat + "\n\n" + "*"*30 + "\n" + feature_glcm
            window['feature_output'].update(feature)
        except Exception as e:
            sg.popup(e)
        
    elif event == "feature_extraction" :
        root_folder = values["Preprocess_Folder"]
        if root_folder == "":
            sg.popup("Please Select Preprocessed Folder First!") # show Popup if Dataset not selected
        else :
            Feature = FeatureExtraction(PREPROCESSED_DATASET_FOLDER = root_folder)
            Feature.ImageRead()
            Feature.CalcStatisticalFeature()
            Feature.CalcGLCMMatrix()
            Feature.CalcGLCMTextureFeature()
            sg.popup("Feature Extraction generated successfully! ") # show Popup to notify feature extrcation run successfully!
            
            # Cleanup window
            window['feature_output'].update("<no feature to show>")
            window['feature_extraction'].update(visible=False)
            window['feature_reset'].update(visible=True)
        
window.close()
del Feature